In [1]:
import os
import shutil
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from datetime import date
from datetime import datetime
from statsmodels.graphics.tsaplots import plot_acf#, plot_pacf, month_plot # => noch pacf nutzen???
from statsmodels.tsa.seasonal import STL

In [2]:
from autots import AutoTS

# Exploration des Datensatzes

Als erstes wird der genutzte Datensatz untersucht. Dabei werden die statistischen Eigenschaften der Zeitreihe untersucht, um gegebenfalls Rückschlüsse auf geeignete Modelle ziehen zu können.

In [3]:
data = pd.read_csv("../data/traffic.csv", parse_dates=True, index_col="DateTime")
data.drop(["ID"], axis=1, inplace=True)
datetime_data = data.copy()
grouped_datetime_data = datetime_data.groupby(by="Junction")

Nachdem laden des Datensatzes wird die Spalte "ID" aus dem Datensatz entfernt. Da es sich um eine pro Datum eindeutige ID handelt hat dieses Attribut keinen Wert für eine Vorhersage.

In [4]:
junction_to_color_mapping = {
    1: "blue",
    2: "green",
    3: "red",
    4: "darkorange",
}

Um die vier Kreuzungen auf Diagrammen unterscheiden zu können wird jeder Kreuzung eine Farbe zugeordnet, die durchgehend verwendet wird.

In [27]:
def create_observation_count_per_junction_barplot(data_grouped_by_junction):
    fig, ax = plt.subplots(figsize=(8,5))
    
    x_values = []
    y_values = []
    color_list = []
    
    for key, group in data_grouped_by_junction:
        x_values.append(key)
        y_values.append(group.size)
        color_list.append(junction_to_color_mapping[key])
        
    ax.bar(x_values, y_values, color=color_list)
    ax.set_xticks(x_values)
    ax.set_xticklabels(
        map(lambda x: "Kreuzung {0}".format(x), x_values)
    )
    #ax.set_title("Number of datapoints available in the traffic dataset per junction", fontsize=15)
    ax.set_xlabel(
        "Vorhandene Kreuzungen im \"traffic prediction\" Datensatz", 
        fontsize=15,
    )
    ax.set_ylabel(
        "Anzahl Datenpunkte pro Kreuzung",
        fontsize=15,
    )
    
    ax.tick_params(labelsize=13)
    
    plt.savefig("./eda_plots/traffic_dataset_datapoint_per_junction_bar_plot.png")
    plt.close(fig)

In [28]:
create_observation_count_per_junction_barplot(grouped_datetime_data)

![datapoints per junction](./eda_plots/traffic_dataset_datapoint_per_junction_bar_plot.png)

Dieses Diagramm zeigt, wie viele Datenpunkte pro Kreuzung im Datensatz vorhanden sind. Es zeigt sich, dass zu Kreuzung vier deutlich weniger Datenpunkte vorliegen als zu den anderen drei Kreuzungen.

In [112]:
def create_bar_plot_by_junction(data_grouped_by_junction):
    fig, axs = plt.subplots(4, 1, figsize=(30,15))
   
    
    for key, group in data_grouped_by_junction:
        vehicle_counts = group["Vehicles"].value_counts().sort_index()
        vehicle_counts.plot(
            kind="bar", 
            ax=axs[key - 1], 
            color=junction_to_color_mapping[key],
        )
        
        #fig.suptitle(
        #    "Distinct vehicle count value occurences per junction of the traffic dataset",
        #    fontsize=20,
        #)
        
        axs[key - 1].set_title(
            "Häufigkeit bestimmter Fahrzeuganzahlen an Kreuzung {0}".format(key), 
            fontsize=25,
        )
        axs[key - 1].set_xlabel(
            "Fahrzeuganzahlwerte", 
            fontsize=20,
        )
        axs[key - 1].set_ylabel(
            "Häufigkeit der \nFahrzeuganzahlwerte", 
            fontsize=20,
        )
        
        axs[key -1].tick_params(labelsize=13)
        axs[key -1].tick_params(axis='x', labelrotation = 45)
        
    fig.tight_layout()
    fig.subplots_adjust(top=0.95)
        
    plt.savefig("./eda_plots/traffic_dataset_vehicle_count_per_junction_bar_plot.png")
    plt.close(fig)

In [113]:
create_bar_plot_by_junction(grouped_datetime_data)

![datapoints per junction](./eda_plots/traffic_dataset_vehicle_count_per_junction_bar_plot.png)

Dieses Balkendigramm zeigt die empirisch gesammelten Häufigkeiten der verschiedenen Fahrzeuganzahlen an allen vier Kreuzungen. Kreuzung eins scheint am meisten befahren zu werden. Die maximale Fahrzeuganzahl an dieser Kreuzung beträgt 156. Fahrzeuganzahlwerte zwischen 5 und ca. 117 kommen regelmäßig vor. Betrachtet man die y-Achsen Skalierung zeigt sich, dass die verschieden Fahrzeuganzahlen weniger Häufig vorkommen, als bei den anderen Kreuzungen.

Kreuzung vier wird von allen Kreuzungen am wenigsten befahren. Dort sind nur Fahrzeuganzahlen von bis zu 20 regelmäßig vertreten. Amhäufigsten sind zwischen 3 und 11 Fahrzeuge an der Kreuzung. Die Tatsache, dass zu dieser Kreuzung am wenigsten Daten vorliegen dürfte keinen Einfluss auf diesen vergleich haben, da sie eher die Skalierung der y als der x-Achse beeinflussen würde.

Sowohl bei Kreuzung zwei und drei liegt der Fahrzeuganzahlwert meistens im Bereich zwischen 1 und 35 Fahrzeugen. Die Skalierung der x-Achse der Diagramme zu diesen beiden Kreuzungen stimmt ebenfalls überein.
Sie unterscheiden sich aber auch in der Hinsicht, dass Kreuzung drei sehr selten auch einen deutlich höhere Fahrzeuganzahlwert aufweist. Der maximale Wert beträgt 180. Das ist der höchste im Datensatz vorhandenen Fahzeuganzahlwert. 

In [116]:
def create_hist_plot_by_junction(data_grouped_by_junction):
    fig, axs = plt.subplots(
        2, 
        2, 
        figsize=(20,15),
        sharex=True,
        sharey=True,
    )
    
    #fig.suptitle(
    #        "Histograms of the vehicle counts of the traffic dataset"
    #        " per junction with identicall x and y axis scale",
    #        fontsize=30,
    #    )
    
    column = 0
    row = 0
    
    for key, group in data_grouped_by_junction:
        group.hist(
            ax=axs[column, row],
            column="Vehicles",
            bins=25,
            grid=False,
            color=junction_to_color_mapping[key],
        )
        
        axs[column, row].xaxis.set_tick_params(labelbottom=True)
        axs[column, row].yaxis.set_tick_params(labelbottom=True)
        axs[column, row].set_title(
            "Histogramm der Häufigkeiten \nFahrzeuganzahlwerte an Kreuzung {0}".format(key),
            fontsize=25,
        )
        axs[column, row].set_xlabel("Fahrzeuganzahlwerte", fontsize=20,)
        axs[column, row].set_ylabel(
            "Häufigkeit der Fahrzeuganzahlwerte \nin festgelegten Wertebereichen",
            fontsize=20,
        )
        
        axs[column, row].tick_params(labelsize=16)
    

        
        column = 0 if key % 2 == 0 else column + 1
        row = row + 1 if column % 2 == 0 else row
        
    fig.tight_layout()
    fig.subplots_adjust(top=0.92)

    plt.savefig("./eda_plots/traffic_dataset_vehicle_count_per_junction_hist.png")
    plt.close(fig)

In [117]:
create_hist_plot_by_junction(grouped_datetime_data)

![datapoints per junction](./eda_plots/traffic_dataset_vehicle_count_per_junction_hist.png)


Mit Hilfe dieser Histogramme wird versucht die wahre Verteilung der Fahrzeuganzahlwerte an den verschiedenen Kreuzungen anzunähern. Die allgemeine Form der Verteilungen ähnelt sich bei allen vier Kreuzungen. Sie unterscheiden sich aber deutlich bezüglich ihrer Stauchung in x-Richtung. Auf Grund dieser Tatsache könnte es sinnvoll sein pro Kreuzung ein eigenes Modell zu trainieren.

In [120]:
def plot_all_junction_timeseries_in_one_plot(data_grouped_by_junction, time_frequence_shown):
        fig, ax = plt.subplots(figsize=(25,12))
        
        frequency_mapping_dict = {
            "H": "Hour",
            "D": "Day",
            "W": "Week",
            "M": "Month",
            "Y": "Year",
        }
        
        for key, group in data_grouped_by_junction:
            group.drop(["Junction"], axis=1, inplace=True)
            resampled_data = group.resample(time_frequence_shown).mean()
            
            ax.plot(
                resampled_data, 
                color = junction_to_color_mapping[key],
                label = "Kreuzung {0}".format(key)
            )
                    
        leg = ax.legend(fontsize="xx-large")

        for line in leg.get_lines():
            line.set_linewidth(6)
        
        #if time_frequence_shown == "H":
        #    ax.set_title(
        #        ("{0}ly vehicle count per junction"        
        #        " in the traffic dataset").format(frequency_mapping_dict["H"]),
        #        fontsize = 25,
        #    )
        #else:
        #    ax.set_title(
        #        ("{0}ly vehicle count per junction"        
        #         " in the traffic dataset calculated as mean of the hourly dates").format(
        #            frequency_mapping_dict[time_frequence_shown]
        #        ),
        #        fontsize = 25,
        #    )
        
        ax.tick_params(labelsize=20)
        ax.tick_params(axis='x', labelrotation = 45)
        
        ax.set_xlabel("Zeitpunkt t", fontsize=25)
        ax.set_ylabel("Anzahl Fahrzeuge zum Zeitpunkt t", fontsize=25)
        
        plt.savefig(
            (
                "./eda_plots/traffic_dataset_combined"
                "_timeseries_plot_{0}_frequency.png"
            ).format(frequency_mapping_dict[time_frequence_shown])
        )

        plt.close(fig)

In [121]:
plot_all_junction_timeseries_in_one_plot(grouped_datetime_data, "D")

![datapoints per junction](./eda_plots/traffic_dataset_combined_timeseries_plot_Day_frequency.png)

Die im letzten Abschnitt durchgeführte Untersuchung der Verteilung der Fahrzeuganzahlwerte war zeitunabhängig.
Um zu prüfen, ob sich die Daten auch in Abhängigkeit zur Zeit deutlich unterscheiden, werden in diesem Diagramm die Zeitreihendaten pro Kreuzung dargestellt. Im Datensatz liegen die Datenpunkte mit einem Abstand von jeweils einer Stunde vor. Um das Diagramm übersichtlicher zu gestalten wurde pro Tag jeweils der Mittelwert der Fahrzeuganzahlwerte dargestellt. 

Diese Darstellung zeigt, dass Kreuzung eins nicht nur am stärksten befahren ist, sondern auch, dass der Verkehr im Zeitraum von November 2015 bis Juli 2017 an dieser Kreuzung am deutlichsten zugenommen hat.

Die Kreuzungen drei und vie scheinen generell keinen Zuwachs an Verkehr verzeichenn zu können. Natürlich sind Tage und Wochen mit besonders viel Verkehr vorhanden, aber auf den gesamten abgebildeten Zeitraum bezogen scheint die Verkehrdichte ungefähr gleich zu bleiben. Gegebenenfalls steigt die Verkehrsdichte an der dritten Kreuzung seit dem Mai 2017, aber es leigen zu wenige Daten vor um das abschließend sagen zu können.

Die Zeitreihe zu zweiten Kreuzung bleibt bis zum Januar 2017 auch eher stationär. Bis auf einzelne Ausnahmen scheint kein steigender oder fallender Trend zu beobachten sein. Ab dem Januar 2017 ändert sich das. Im Zeitraum zwischen Januar und Juli 2017 scheint das Verkehrsaufkommen an dieser Kreuzung zu steigen.

Dieses Diagramm zeigt, dass es notwendig ist jeweils verschiedenen Modelle pro Kreuzung zu testen.

In [124]:
def plot_all_junction_timeseries_decomposition_in_one_plot(data_grouped_by_junction, time_frequence_shown):
    fig, axs = plt.subplots(
        nrows = 4, 
        ncols = 1, 
        figsize=(30,18),
    )
    
    frequency_mapping_dict = {
        "H": "Hour",
        "D": "Day",
        "W": "Week",
        "M": "Month",
        "Y": "Year",
    }
    
    for key, group in data_grouped_by_junction:
        group.drop(["Junction"], axis=1, inplace=True)
        resampled_data = group.resample(time_frequence_shown).mean()
        decomposed_ts = STL(resampled_data).fit()
        
        axs[0].plot(
            decomposed_ts.observed,
            color = junction_to_color_mapping[key],
            label = "Kreuzung {0}".format(key),
        )
        axs[1].plot(
            decomposed_ts.trend,
            color = junction_to_color_mapping[key],
        )
        axs[2].plot(
            decomposed_ts.seasonal,
            color = junction_to_color_mapping[key],
        )
        axs[3].plot(
            decomposed_ts.resid,
            color = junction_to_color_mapping[key],
        )
        
        
    #if time_frequence_shown == "H":
    #        fig.suptitle(
    #            ("Decomposition of the {0}ly vehicle count data per junction"        
    #            " in the traffic dataset").format(frequency_mapping_dict["H"]),
    #            fontsize = 30,
    #            y = 0.92,
    #        )
    #else:
    #    fig.suptitle(
    #        ("Decomposition of the {0}ly vehicle count data per junction"        
    #         " in the traffic dataset calculated as mean of the hourly dates").format(
    #            frequency_mapping_dict[time_frequence_shown]
    #        ),
    #        fontsize = 30,
    #        y = 0.92,
    #    )
    
    axs[0].set_title("Beobachtete Zeitreihendaten pro Kreuzung", fontsize=24)
    axs[1].set_title("Trend Komponente der Zeitreihendaten pro Kreuzung", fontsize=24)
    axs[2].set_title("Saison Komponente der Zeitreihendaten pro Kreuzung", fontsize=24)
    axs[3].set_title("Irreguläre Komponente der Zeitreihendaten pro Kreuzung", fontsize=24)
    
    #axs[0].set_ylabel("Fahrzeuganzahl zum \nZeitpunkt t", fontsize=20)
    #axs[1].set_ylabel("Fahrzeuganzahl zum \nZeitpunkt t", fontsize=20)
    #axs[2].set_ylabel("Fahrzeuganzahl zum \nZeitpunkt t", fontsize=20)
    axs[3].set_xlabel("Zeitpunkt t der Datenaufzeichnung", fontsize=20)
    #axs[3].set_ylabel("Fahrzeuganzahl zum \nZeitpunkt t", fontsize=20)
    
    axs[0].tick_params(labelsize=20)
    axs[1].tick_params(labelsize=20)
    axs[2].tick_params(labelsize=20)
    axs[3].tick_params(labelsize=20)
        
    leg = fig.legend(
        fontsize="xx-large", 
        ncol = 2,
        bbox_to_anchor =(0.6, 0.095)
    )
    
    for line in leg.get_lines():
            line.set_linewidth(6)
            
    # y-achsen beschriftung anpassen!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
            
    fig.tight_layout(h_pad=2.0)
    fig.subplots_adjust(bottom=0.15)
    
    plt.savefig(
        "./eda_plots/timeseries_decomposition_per_junction.png",
        facecolor = "w",
    )
    
    plt.close(fig)

In [125]:
plot_all_junction_timeseries_decomposition_in_one_plot(grouped_datetime_data, "D")

![datapoints per junction](./eda_plots/timeseries_decomposition_per_junction.png)

Da es sich bei dem vorliegenden Datensatz um Univariate Zeitreihendaten handelt, können Vorhersagen über zukünftige Werte nur auf Grundlage der vergangenen Werte getroffen werden. Um den der Zeitreihe zu Grunde liegenden stochastischen Prozess anzunähern muss eine Funktion an Hand der vorliegenden Daten so angepasst werden, dass mit den p letzten Datenpunkten als Eingabe eine möglichst gute Vorhersage für den Zeitpunkt t + 1 als Ausgabe liefert.
Von einfachen Methoden wie dem Durchschnitt der letzten p Datenpunkte bis zu Neuronalen Netzen gibt es verschieden komplexe Modelle um die wahre erzeugende Funktion anzunähern.
Je komplexer diese wahre Funktion ist, desto komplexer müssen die Modelle sein, mit deren Hilfe sie angenähert werden soll. Komplexität bedeutet in diesem Kontext, dass das Verhalten zum Zeitpunkt t + 1 keinem einfachen Muster folgt. Ein einfaches Muster wäre in diesem Kontext z.B. wenn sich der Fahrzeuganzahlwert alle sieben Tage mit einer Abweichung von maximal +- zwei Fahrezeugen gleicht.

Eine Dekomposition der Zeitreihendaten in einen generellen Trend Anteil, einen Saisonalen Anteil und einen Irregulären Anteil kann so Aufschluss darüber geben, welche Art von Modellen geeignet seien könnten. Natürlich ist dies kein definitiver Beweis für die Eignung eines bestimmten Modells.
Diese Dekomposition folgt entweder einem additiven oder einem multiplikativen Modell.
Im Falle eines additiven Modells, ergibt sich der wahre Fahrzeuganzahlwert zum Zeitpunkt t folgendermaßen:

$Fahrzeuganzahlwert_t = {Trend Anteil}_t + {Saisonaler Anteil}_t + {Irregulärer Anteil}_t$

Dementsprechend sieht der Ansatz eines multiplikativen Modells wie folgt aus:

$Fahrzeuganzahlwert_t = {Trend Anteil}_t \cdot {Saisonaler Anteil}_t \cdot {Irregulärer Anteil}_t$

Das obige Diagramm zeigt eine additive Dekomposition für alle vier Kreuzungen. Die Datenpunkte wurden wieder durch Mittelwertbildung auf eine tägliche Frequenz reduziert. Die Beobachtungen, die an Hand des vorherigen Zeitreihendiagramms bezüglich der generellen Entwicklung der Verkehrsdichte gemacht wurden, werden von den aus diesem Diagramm ablesbaren Trend Komponenten der Zeitreihendaten pro Kreuzung unterstützt.

Auch die Beobachtung der wachsenden Vekehrsdichte an Kreuzung eins über den aufgezeichneten Zeitraum ist auf der Darstellung der Saisonalen Komponentze zu sehen.Diese Darstellung zeigt auch, dass die Vekehrsdichte an Kreuzung zwei ebenfalls über den aufgezeichneten Zeitraum kontinuierlich steigt. Diese Tatsache war im Diagramm der kompletten Zeitreihe nicht so deutlich abzulesen.

Die Darstellung der Irregulären Komponente zeigt, dass vorallem an Kreuzung eins und drei verhältnissmäßig oft ein nicht zu vernachlässigender Anteil des Fahrzeuganzahlwertes nicht auf den kombinierten Trend und Saison Anteil zurückzuführen ist.

Insgesamt lässt dieses Diagramm die Vermutung zu, dass für die Kreuzungen zwei und vier gegebenenfalls einfacherere Modelle ausreichen. Da ein Großteil des Fahrzeuganzahlwertes nur auf dem Trend und dem Saisonalen Komponenten beruht, muss das Modell wahrscheinlich keine allzu komplexe Funktion annähern.
Die teilweise großen Werte des Irregulären Anteils bei den Kreuzungen eins und drei lässt vermuten, dass die Modelle, die diese Zeitreihe annähern unter Umständen deutlich komplizierter seien müssen, oder die trainierten Modelle für diese Kreuzungen keine so zuverlässigen Werte vorhersagen werden.

In [149]:
def plot_autocorrelation_by_junctions(
    data_grouped_by_junction,
    frequency,
    lag,
):
    fig, axs = plt.subplots(
        2, 
        2, 
        figsize=(20,10),
    )
    
    #fig.suptitle(
     #       "Histograms of the vehicle counts of the traffic dataset"
     #       " per junction with identicall x and y axis scale",
      #      fontsize=20,
     #   )
    
    frequency_mapping_dict = {
        "H": "Stunde",
        "D": "Tag",
        "W": "Woche",
        "M": "Monat",
        "Y": "Jahr",
    }
    
    frequency_mapping_dict_plural = {
        "H": "Stunden",
        "D": "Tage",
        "W": "Wochen",
        "M": "Monate",
        "Y": "Jahre",
    }
    
    
    
    column = 0
    row = 0
    
    for key, group in data_grouped_by_junction:
        #axs[column, row].set_title(
        #    "Histogram for vehicle count data at junction {0}".format(key),
        #    fontsize=18,
        #)
        
        resampled_data = group.resample(frequency).mean()

        plot_acf(
            group["Vehicles"], 
            lags=lag,
            ax=axs[column, row],
            auto_ylims=True,
            vlines_kwargs = {"colors": junction_to_color_mapping[key]},
            color=junction_to_color_mapping[key],
        )
        
        axs[column, row].set_title(
            "Autokorrelation der Zeitreihe an Kreuzung {0}".format(key),
            fontsize = 25,
        )
        axs[column, row].set_ylabel("Autokorrelationswert", fontsize=20)
        axs[column, row].set_xlabel(
            "Bezugszeitpunkt {0} t - {1} {2}".format(
                frequency_mapping_dict[frequency],
                lag,
                frequency_mapping_dict_plural[frequency],
            ), 
            fontsize=20,
        )

        axs[column, row].tick_params(labelsize=15)
        
        column = 0 if key % 2 == 0 else column + 1
        row = row + 1 if column % 2 == 0 else row
        
    fig.tight_layout()
    fig.subplots_adjust(top=0.95)
    
    plt.savefig(
        "./eda_plots/traffic_dataset_autocorrelation_per_junction_plot_{0}_{1}.png".format(
            frequency,
            lag,
        )
    )
    plt.close(fig)

In [150]:
plot_autocorrelation_by_junctions(grouped_datetime_data, "D", 7)
plot_autocorrelation_by_junctions(grouped_datetime_data, "D", 30)
plot_autocorrelation_by_junctions(grouped_datetime_data, "M", 12)

![datapoints per junction](./eda_plots/traffic_dataset_autocorrelation_per_junction_plot_D_7.png)


![datapoints per junction](./eda_plots/traffic_dataset_autocorrelation_per_junction_plot_D_30.png)


In [ ]:
# => stat test ob statisch? => hätte das implikationen oder kann das durch preprocessing egal gemacht werden????

# Training

In [16]:
# metric_weighting: dict = {
#            'smape_weighting': 5,
#            'mae_weighting': 2,
#            'rmse_weighting': 2,
#            'made_weighting': 0.5,
#            'mage_weighting': 0,
#            'mle_weighting': 0,
#            'imle_weighting': 0,
#            'spl_weighting': 3,
#            'containment_weighting': 0,
#            'contour_weighting': 1,
#            'runtime_weighting': 0.05,
#            'oda_weighting': 0.001,
#        },

In [151]:
def prepare_filesystem_for_saving_results():
    for junction_number in range(1, 5):
        junction_result_folder = "./results_per_junction/junction_{0}".format(junction_number) 
        
        if not os.path.exists(junction_result_folder):
            os.makedirs(junction_result_folder)
            print("Directory {0} succesfully created.".format(junction_result_folder))
        else:    
            print("Directory {0} already exists.".format(junction_result_folder))
            
    for junction_result_folder in os.listdir("./results_per_junction"):
        try:
            os.mkdir("./results_per_junction/{0}/autots_model".format(junction_result_folder))
            os.mkdir("./results_per_junction/{0}/best_quarter_of_trained_models".format(junction_result_folder))
            os.mkdir("./results_per_junction/{0}/result_plots".format(junction_result_folder))
            os.mkdir(
                "./results_per_junction/{0}/"
                "result_tables_as_latex".format(junction_result_folder)
            )
            print(
                "Directories ./results_per_junction/{0}/autots_model,"
                " ./results_per_junction/{0}/result_plots,"
                " ./results_per_junction/{0}/best_quarter_of_trained_models"
                "and ./results_per_junction/{0}/result_tables_as_latex"
                " successfully created.".format(junction_result_folder)
            )
        except FileExistsError:
            print(
                "Directories ./results_per_junction/{0}/autots_model,"
                " ./results_per_junction/{0}/result_plots,"
                " ./results_per_junction/{0}/best_quarter_of_trained_models"
                "and ./results_per_junction/{0}/result_tables_as_latex" 
                " already existed.".format(junction_result_folder)
            )  
    
    try:
        os.mkdir("./combined_result_plots")
        print("Directory ./combined_result_plots successfully created.")
    except FileExistsError:
        print("Directory ./combined_result_plots already existed")  
    

def cleanup_filesystem():
    try:
        shutil.rmtree("./combined_result_plots")
    except OSError as e:
        print("Error: {0} : {1}".format("./combined_result_plots", e.strerror))
        
    try:
        shutil.rmtree("./results_per_junction")
    except OSError as e:
        print("Error: {0} : {1}".format("./results_per_junction", e.strerror))


def save_autots_model_results_per_single_junction(
    autots_model_single_junction,
    junction_key,
):
    filename = ("junction_{0}_trained_autots_model_junction_"       
                    "number_tuple.bin").format(junction_key)
        
    working_directory =os.getcwd()  # da lassen? eig nicht nötig?!
    base_path = "results_per_junction/junction_{0}/autots_model".format(junction_key)
        
    complete_path = "/".join([working_directory, base_path, filename])
        
    with open(complete_path, "wb") as f: # "wb" because we want to write in binary mode
        pickle.dump((autots_model_single_junction, junction_key), f)
            
            
        
def save_best_model_per_junctions_backforecast(
    best_model_single_junction,
    junction_num,
    #time_series_data_per_junction,
):
    filename = ("junction_{0}_best_trained_model_junction_"       
                    "backforecast.csv").format(junction_num)
        
    working_directory =os.getcwd()  # da lassen? eig nicht nötig?!
    base_path = "results_per_junction/junction_{0}/autots_model".format(junction_num)
        
    complete_path = "/".join([working_directory, base_path, filename])
                
    junction_backforecast = best_model_single_junction.back_forecast(
        column = "Vehicles",
        n_splits = "auto",
    ).forecast
        
    junction_backforecast.to_csv(complete_path)
        


def train_models_per_junction(data_grouped_by_junction, autots_param_dict):
    result_model_junction_key_list = []
    
    for key, group in data_grouped_by_junction:
        group.drop(["Junction"], axis=1, inplace=True)
        group["Vehicles"] = group["Vehicles"].astype("float64")

        autots_model = AutoTS(**autots_param_dict)
        
        junction_resulting_model = autots_model.fit(group)
        
        save_autots_model_results_per_single_junction(
            junction_resulting_model, 
            key
        )
        
        save_best_model_per_junctions_backforecast(
            junction_resulting_model, 
            key,
        )
        
        result_model_junction_key_list.append((junction_resulting_model, key))
        
    return result_model_junction_key_list    

In [152]:
model_dict = {
    'ConstantNaive': 1,
    'LastValueNaive': 1.5,
    'AverageValueNaive': 1,
    'GLS': 1,
    'SeasonalNaive': 1,
    'GLM': 1,
    'ETS': 1,
    'UnobservedComponents': 0.4,  # it's fast enough but I'll leave for parallel
    'WindowRegression': 0.3,  # this gets slow with Transformer, KerasRNN
    'DatepartRegression': 0.5,
    'UnivariateMotif': 1,
    'SectionalMotif': 1,
    'NVAR': 1,
    'FBProphet': 1,
}
    

autots_param_dict = {
    "forecast_length": 5,
    "frequency": "H",
    "prediction_interval": 0.9,
    "ensemble": None,
    "model_list": "superfast",
    "transformer_list": "superfast",
    "max_generations": 5,
    "num_validations": 5,
    "validation_method": "backwards",
    "n_jobs": 2,
    "no_negatives": True,
    "holiday_country": "US",
}


#cleanup_filesystem()

prepare_filesystem_for_saving_results()

#resulting_models = train_models_per_junction(grouped_datetime_data, autots_param_dict)

Directory ./results_per_junction/junction_1 succesfully created.
Directory ./results_per_junction/junction_2 succesfully created.
Directory ./results_per_junction/junction_3 succesfully created.
Directory ./results_per_junction/junction_4 succesfully created.
Directories ./results_per_junction/junction_4/autots_model, ./results_per_junction/junction_4/result_plots, ./results_per_junction/junction_4/best_quarter_of_trained_modelsand ./results_per_junction/junction_4/result_tables_as_latex successfully created.
Directories ./results_per_junction/junction_2/autots_model, ./results_per_junction/junction_2/result_plots, ./results_per_junction/junction_2/best_quarter_of_trained_modelsand ./results_per_junction/junction_2/result_tables_as_latex successfully created.
Directories ./results_per_junction/junction_1/autots_model, ./results_per_junction/junction_1/result_plots, ./results_per_junction/junction_1/best_quarter_of_trained_modelsand ./results_per_junction/junction_1/result_tables_as_lat

In [156]:
data_junction_3 = [(3, grouped_datetime_data.get_group(3))]

print(data_junction_3)

[(3,                      Junction  Vehicles
DateTime                               
2015-11-01 00:00:00         3         9
2015-11-01 01:00:00         3         7
2015-11-01 02:00:00         3         5
2015-11-01 03:00:00         3         1
2015-11-01 04:00:00         3         2
...                       ...       ...
2017-06-30 19:00:00         3        33
2017-06-30 20:00:00         3        31
2017-06-30 21:00:00         3        28
2017-06-30 22:00:00         3        26
2017-06-30 23:00:00         3        39

[14592 rows x 2 columns])]


In [157]:
single_junc_res = train_models_per_junction(data_junction_3, autots_param_dict)

Model Number: 1 with model AverageValueNaive in generation 0 of 5
Model Number: 2 with model AverageValueNaive in generation 0 of 5


/home/paettfish/.local/share/virtualenvs/experiments-mVQQwhcP/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/tmp/ipykernel_1392071/1104939445.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group["Vehicles"] = group["Vehicles"].astype("float64")


Model Number: 3 with model AverageValueNaive in generation 0 of 5
Model Number: 4 with model GLS in generation 0 of 5
Model Number: 5 with model GLS in generation 0 of 5
Model Number: 6 with model LastValueNaive in generation 0 of 5
Model Number: 7 with model LastValueNaive in generation 0 of 5
Model Number: 8 with model LastValueNaive in generation 0 of 5
Model Number: 9 with model LastValueNaive in generation 0 of 5
Model Number: 10 with model SeasonalNaive in generation 0 of 5
Model Number: 11 with model SeasonalNaive in generation 0 of 5
Model Number: 12 with model SeasonalNaive in generation 0 of 5
Model Number: 13 with model ConstantNaive in generation 0 of 5
Model Number: 14 with model SeasonalNaive in generation 0 of 5
Model Number: 15 with model SeasonalNaive in generation 0 of 5
Model Number: 16 with model ConstantNaive in generation 0 of 5
Model Number: 17 with model LastValueNaive in generation 0 of 5
Model Number: 18 with model AverageValueNaive in generation 0 of 5
Model 

Model Number: 134 with model AverageValueNaive in generation 4 of 5
Model Number: 135 with model SeasonalNaive in generation 4 of 5
Model Number: 136 with model LastValueNaive in generation 4 of 5
Model Number: 137 with model LastValueNaive in generation 4 of 5
Model Number: 138 with model AverageValueNaive in generation 4 of 5
Model Number: 139 with model SeasonalNaive in generation 4 of 5
Model Number: 140 with model GLS in generation 4 of 5
New Generation: 5 of 5
Model Number: 141 with model LastValueNaive in generation 5 of 5
Model Number: 142 with model LastValueNaive in generation 5 of 5
Model Number: 143 with model LastValueNaive in generation 5 of 5
Model Number: 144 with model GLS in generation 5 of 5
Model Number: 145 with model AverageValueNaive in generation 5 of 5
Model Number: 146 with model LastValueNaive in generation 5 of 5
Model Number: 147 with model SeasonalNaive in generation 5 of 5
Model Number: 148 with model GLS in generation 5 of 5
Model Number: 149 with model 

12 - SeasonalNaive with avg smape 72.85: 
Model Number: 13 of 24 with model ConstantNaive for Validation 3
13 - ConstantNaive with avg smape 70.15: 
Model Number: 14 of 24 with model AverageValueNaive for Validation 3
14 - AverageValueNaive with avg smape 78.24: 
Model Number: 15 of 24 with model GLS for Validation 3
📈 15 - GLS with avg smape 24.16: 
Model Number: 16 of 24 with model ConstantNaive for Validation 3
16 - ConstantNaive with avg smape 78.26: 
Model Number: 17 of 24 with model ConstantNaive for Validation 3
17 - ConstantNaive with avg smape 78.26: 
Model Number: 18 of 24 with model ConstantNaive for Validation 3
18 - ConstantNaive with avg smape 78.26: 
Model Number: 19 of 24 with model AverageValueNaive for Validation 3
19 - AverageValueNaive with avg smape 78.27: 
Model Number: 20 of 24 with model AverageValueNaive for Validation 3
20 - AverageValueNaive with avg smape 78.27: 
Model Number: 21 of 24 with model AverageValueNaive for Validation 3
21 - AverageValueNaive with

In [107]:
def load_trained_models_from_file():
    base_dir = "./results_per_junction"
    generic_model_dirs = "junction_{0}"
    generic_fn = "junction_{0}_trained_autots_model_junction_number_tuple.bin"
    
    trained_model_junction_num_tuple_list = []
    
    for junction_number in range(1, 5):
        filepath = "/".join(
            [
                base_dir, 
                generic_model_dirs.format(junction_number), 
                "autots_model", 
                generic_fn.format(junction_number)
            ]
        )
                
        with open(filepath, "rb") as f: # "wb" because we want to write in binary mode
            trained_model_junction_num_tuple = pickle.load(f)  
            
        trained_model_junction_num_tuple_list.append(
            trained_model_junction_num_tuple
        )
        
    return trained_model_junction_num_tuple_list


def create_best_model_scores_per_junction_bar_plot(best_model_junction_num_tuple_list):
    fig, ax = plt.subplots(figsize=(8,5))
    
    score_list = []
    junction_num_list = []
    color_list = []
    
    for autots_model, junction_num in best_model_junction_num_tuple_list:
        model_results = autots_model.results("validation")
        
        best_model_index = autots_model.best_model.index
        
        score = model_results.iloc[best_model_index]["Score"].values
        
        score_list.append(*score)
        junction_num_list.append(junction_num)
        color_list.append(junction_to_color_mapping[junction_num])
        
    ax.bar(junction_num_list, score_list, color=color_list)
    
    ax.set_xticks(junction_num_list)
    ax.set_xticklabels(
        map(lambda x: "junction_{0}".format(x), junction_num_list)
    )
    #ax.set_title("Number of datapoints available in the traffic dataset per junction", fontsize=15)
    #ax.set_xlabel("Junctions present in the traffic dataset")
    #ax.set_ylabel("Number of available datapoints")
    
    #plt.savefig("./eda_plots/traffic_dataset_datapoint_per_junction_bar_plot.png")
    #plt.close(fig)
    

def create_model_count_in_best_worst_quarter_of_models_per_junction_bar_plot(
    best_model_junction_num_tuple_list
):
    for model, junction_num in best_model_junction_num_tuple_list:
        results = model.results("validation")
        val_results = results.loc[results["Runs"] == 6].sort_values(by=["Score"])
        
        n_quarter = round(len(val_results) / 4)
        
        quarter_best_models = val_results.head(n_quarter)[["Model", "ID"]]
        quarter_worst_models = val_results.tail(n_quarter)[["Model", "ID"]]
        
        quarter_best_model_counts = quarter_best_models.groupby(["Model"]).count()
        quarter_worst_model_counts = quarter_worst_models.groupby(["Model"]).count()
        
        fig, axs = plt.subplots(
            1, 
            2, 
            figsize=(15,7),
        )
        
        fig.suptitle(
            (
                "Specific model count in best and worst"      
                " performing models quarter on junction_{0} data"
            ).format(junction_num)
        )
        
        axs[0].bar(
            quarter_best_model_counts.index, 
            quarter_best_model_counts["ID"],
            color = junction_to_color_mapping[junction_num],
        )
        axs[0].set_title("Model count in best performing quarter of trained models")


        axs[1].bar(
            quarter_worst_model_counts.index,
            quarter_worst_model_counts["ID"],
            color = junction_to_color_mapping[junction_num],
        )
        axs[1].set_title("Model count in worst performing quarter of trained models")
        
        # y-achse beschriften!!!!! ggf. auch x?!?!?!
        
        #plt.savefig()
        #plt.close(fig)

        
        
def create_model_count_in_best_worst_quarter_of_models_bar_plot(
    best_model_junction_num_tuple_list
):
    all_junctions_best_quarter_models = pd.DataFrame()
    all_junctions_worst_quarter_models = pd.DataFrame()

    
    for model, junction_num in best_model_junction_num_tuple_list:
        results = model.results("validation")
        val_results = results.loc[results["Runs"] == 6].sort_values(by=["Score"])
        
        n_quarter = round(len(val_results) / 4)
        
        quarter_best_models = val_results.head(n_quarter)[["Model", "ID"]]
        quarter_worst_models = val_results.tail(n_quarter)[["Model", "ID"]]
        
        all_junctions_best_quarter_models = pd.concat(
            [
                all_junctions_best_quarter_models,
                quarter_best_models,
            ]
        )
        
        all_junctions_worst_quarter_models = pd.concat(
            [
                all_junctions_worst_quarter_models,
                quarter_worst_models,
            ]
        )
    
    quarter_best_model_counts = all_junctions_best_quarter_models.groupby(["Model"]).count()
    quarter_worst_model_counts = all_junctions_worst_quarter_models.groupby(["Model"]).count()
        
    fig, axs = plt.subplots(
        1, 
        2, 
        figsize=(15,7),
    )
        
    fig.suptitle(
        (
            "Specific model count in best and worst"      
            " performing models quarter combined from all junctions" 
        ).format(junction_num),
        fontsize = 18,
    )
        
    axs[0].bar(
        quarter_best_model_counts.index, 
        quarter_best_model_counts["ID"],    
        color = "silver",    
    )
        
    axs[0].set_title(
        "Model count in best performing quarter of trained models",
        fontsize = 14,
    )

    axs[1].bar(
        quarter_worst_model_counts.index,
        quarter_worst_model_counts["ID"],
        color = "silver",    
    )
    axs[1].set_title(
        "Model count in worst performing quarter of trained models",
        fontsize = 14,
    )
    
    plt.savefig("./combined_result_plots/model_count_in_best_worst_quarter_of_models_bar_plot.png")
    #plt.close(fig)



    
def create_backforecast_plot(
    time_series_data,
    best_model_junction_num_tuple_list, 
    start_date = datetime(2015, 11, 1, 0, 0, 0)
):
    fig, axs = plt.subplots(
        2, 
        2, 
        figsize=(20,15),
    )
    
    column = 0
    row = 0
    
    time_series_data_groups = time_series_data.groupby(["Junction"])
    
    for model, junction_num in best_model_junction_num_tuple_list:
        filename = ("junction_{0}_best_trained_model_junction_"       
                    "backforecast.csv").format(junction_num)
        
        working_directory =os.getcwd()  # da lassen? eig nicht nötig?!
        base_path = "results_per_junction/junction_{0}/autots_model".format(junction_num)
        
        complete_path = "/".join([working_directory, base_path, filename])
        
        junction_backforecast = pd.read_csv(
            complete_path, 
            parse_dates=True,
            header=0,
            names=["DateTime", "Vehicles"],
            index_col="DateTime",
        )
        
        junction_data = time_series_data_groups.get_group(junction_num)["Vehicles"]
                
        reduced_forecast = junction_backforecast[start_date:]
        reduced_data = junction_data[start_date:]
        
        axs[column, row].plot(
            reduced_data, 
            color = junction_to_color_mapping[junction_num],
            label = "junction_{0}".format(junction_num)
        )
        
        axs[column, row].plot(
            reduced_forecast,
            color = "black",
            label = "model_forecast" if junction_num ==1 else None
        )
        
        axs[column, row].set_title(
            "Kreuzung {0}".format(junction_num),
            fontsize = 22,
        )
        
        axs[column, row].tick_params(labelsize=16)
        axs[column, row].tick_params(axis='x', labelrotation = 45)
        
        column = 0 if junction_num % 2 == 0 else column + 1
        row = row + 1 if column % 2 == 0 else row
        
    leg = fig.legend(
        fontsize="xx-large", 
        ncol = 2,
        bbox_to_anchor =(0.65, 0.095)
    )
    
    for line in leg.get_lines():
            line.set_linewidth(6)
            
    fig.tight_layout(h_pad=2.0)
    fig.subplots_adjust(bottom=0.175)
    
    fig.savefig(
        "./combined_result_plots/backforecast_per_junction_plot.png",
        facecolor = "w",
    )
    
    #plt.close(fig)

    
def save_best_quarter_of_models_per_junction(best_model_junction_num_tuple_list):
    generic_path = (
        " ./results_per_junction/{0}/"
        "best_quarter_of_trained_models/"
        "models.csv"
    )
    
    for model, junction_num in best_model_junction_num_tuple_list:
        results = model.results("validation")
        val_results = results.loc[results["Runs"] == 6].sort_values(by=["Score"])
        
        n_quarter = round(len(val_results) / 4)
        
        model.export_template(
            generic_path.format(junction_num), 
            models="best",          
            n=n_quarter, 
        )
        
        
def safe_model_occurences_in_best_quarter_as_latex_table_per_junction(
    best_model_junction_num_tuple_list
):
    working_directory =os.getcwd() # notwendig? denke nicht!
    generic_base_path = "/".join(
        [
            working_directory,
            "results_per_junction/junction_{0}/result_tables_as_latex/"
        ]
    )
    fn_best = "model_occurence_best_quarter_junction_{0}.tex"
    fn_worst = "model_occurence_worst_quarter_junction_{0}.tex"
    
    for model, junction_num in best_model_junction_num_tuple_list:
        results = model.results("validation")
        val_results = results.loc[results["Runs"] == 6].sort_values(by=["Score"])
        
        n_quarter = round(len(val_results) / 4)
        
        quarter_best_models = val_results.head(n_quarter)[["Model", "ID"]]
        quarter_worst_models = val_results.tail(n_quarter)[["Model", "ID"]]
        
        quarter_best_model_counts = quarter_best_models.groupby(["Model"]).count()
        quarter_worst_model_counts = quarter_worst_models.groupby(["Model"]).count()
        
        quarter_best_model_counts.columns = ["occurence"]
        quarter_worst_model_counts.columns = ["occurence"]
 

        best_path = "".join(
            [
                generic_base_path.format(junction_num),
                fn_best.format(junction_num)
            ]
        )
    
        worst_path = "".join(
            [
                generic_base_path.format(junction_num),
                fn_worst.format(junction_num)
            ]
        )
        
        quarter_best_model_counts.to_latex(
            buf = best_path,
            header = True,
            index = False,
            longtable = True,
            #caption = ,
        )
        
        quarter_worst_model_counts.to_latex(
            buf = worst_path,
            header = True,
            index = False,
            longtable = True,
            #caption = ,
        )
        

        
def safe_complete_autots_results_as_latex_table_per_junction(
    best_model_junction_num_tuple_list
):
    generic_base_path = "./results_per_junction/junction_{0}/result_tables_as_latex/"
    
    fn_train_results = "autots_model_train_results_junction_{0}.tex"
    fn_val_results = "autots_model_val_results_junction_{0}.tex"
    
    
    for model, junction_num in best_model_junction_num_tuple_list:
        train_results = model.results()
        val_results = model.results("validation")
        
        train_path = "".join(
            [
                generic_base_path.format(junction_num),
                fn_train_results.format(junction_num),
            ]
        )
        
        val_path = "".join(
            [
                generic_base_path.format(junction_num),
                fn_val_results.format(junction_num),
            ]
        )
        
        
        train_results.to_latex(
            buf = train_path,
            header = True,
            index = False,
            longtable = True,
            #caption = ,
        )
        
        val_results.to_latex(
            buf = val_path,
            header = True,
            index = False,
            longtable = True,
            #caption = ,
        )

        
    
def create_report_materials_out_of_trained_models():
    trained_model_junction_num_tuple_list = load_trained_models_from_file()
    
    create_best_model_scores_per_junction_bar_plot(  # => funk nutzen für tabelle!!!!!!!!!!!!!!!!!!!
        trained_model_junction_num_tuple_list
    )
    
    
    #create_model_count_in_best_worst_quarter_of_models_per_junction_bar_plot(
    #    trained_model_junction_num_tuple_list
    #)
    
    #create_model_count_in_best_worst_quarter_of_models_bar_plot(
    #    trained_model_junction_num_tuple_list
    #)
    
    #safe_model_occurences_in_best_quarter_as_latex_table_per_junction(
    #    trained_model_junction_num_tuple_list
    #)
    
    
    #safe_complete_autots_results_as_latex_table_per_junction(
    #    trained_model_junction_num_tuple_list
    #)
    
    #create_backforecast_plot(
    #    data,
    #    trained_model_junction_num_tuple_list, 
    #    datetime(2017, 6, 15, 0, 0, 0)
    #)
    
    
        
    # avg score by model/ model group
    
    # ggf. generation loss per junction plot????

In [108]:
create_report_materials_out_of_trained_models()

FileNotFoundError: [Errno 2] No such file or directory: './results_per_junction/junction_1/autots_model/junction_1_trained_autots_model_junction_number_tuple.bin'

fzgsdhgfsdghfhjsdghfjsdgfsdgf

sdhfjksdhfkjsdhhfjksdhfkjsd

![datapoints per junction](./eda_plots/traffic_dataset_vehicle_count_per_junction_bar_plot.png)

sahjdgasjhghdhjasgdhjasgd

ashdsahdkjsahdkjsahd

![datapoints per junction](./eda_plots/traffic_dataset_vehicle_count_per_junction_bar_plot.png)



ashndashjkdhasjkdh

asghdjasgdgshajdgh

![datapoints per junction](./eda_plots/traffic_dataset_vehicle_count_per_junction_bar_plot.png)


shakjdhukashdkjashdjkas

jsakhdjhsadkjhaskdh

![datapoints per junction](./eda_plots/traffic_dataset_vehicle_count_per_junction_bar_plot.png)


sdjadfukdshfjksdhfjksdh

sakdhaskjdhasjkhjdkjash

![datapoints per junction](./eda_plots/traffic_dataset_vehicle_count_per_junction_bar_plot.png)


jkdshfjkdshjfsdhfsdfhj

hdsfhsdkjfhsdjkhfjksdhf

![datapoints per junction](./eda_plots/traffic_dataset_vehicle_count_per_junction_bar_plot.png)


In [ ]:
def create_best_model_scores_per_junction_table(best_model_junction_num_tuple_list):
    score_list = []
    junction_num_list = []
    model_name_list = []
    
    for autots_model, junction_num in best_model_junction_num_tuple_list:
        model_results = autots_model.results("validation")
        
        best_model_index = autots_model.best_model.index
        
        score = model_results.iloc[best_model_index]["Score"].values
        
        score_list.append(*score)
        junction_num_list.append(junction_num)
        model_name_list.append(autots_model.best_model_name)
        
    best_model_result_table = pd.DataFrame(
        {
            "Kreuzung": junction_num_list,
            "Modell": model_name_list,
            "Zielmetrikwert": score_list,
        }
    )
    
    return best_model_result_table

In [ ]:
trained_model_junction_num_tuple_list = load_trained_models_from_file()
    
result_table = create_best_model_scores_per_junction_table(
        trained_model_junction_num_tuple_list
    )
    
from IPython.display import HTML
HTML(result_table.to_html(index=False))

In [ ]:
# median runtime per algo
# latex und csv tabellen abspeichern